In [ ]:
df=spark.read.format('csv').option('header',True).load('s3://deloittesalarybucket/multiplesmalldataset/')

Calculation started (calculation_id=74c72ac8-4151-a932-1d9d-f66cdb8696c6) in (session=66c72ac8-2423-fcad-e8fe-1b8214841318). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [ ]:
df.count()

Calculation started (calculation_id=ccc72ac9-94e6-4e59-4a91-82a37be61c0d) in (session=66c72ac8-2423-fcad-e8fe-1b8214841318). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
17002



In [ ]:
import boto3
import json
import math
session = boto3.Session(region_name='us-east-1')
s3_client = session.client('s3')

Calculation started (calculation_id=40c72ac8-6d3f-6eca-23d7-2c0a13b3c401) in (session=66c72ac8-2423-fcad-e8fe-1b8214841318). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [ ]:
def get_partition_num_and_size(bucket, prefix):
    keys = []
    list_objects_v2_paginator = s3_client.get_paginator('list_objects_v2')
    for s3_list_page in list_objects_v2_paginator.paginate(Bucket=bucket, Prefix=prefix):
        keys.extend(s3_list_page['Contents'])
    partition_size_in_bytes = 0
    partition_file_num = 0
    for key in keys:
        if not key['Key'].endswith('/'):
            file_size_in_bytes = int(key['Size'])
            partition_size_in_bytes += file_size_in_bytes
            partition_file_num += 1

    return partition_file_num, partition_size_in_bytes

Calculation started (calculation_id=02c72ac8-7293-7716-a104-ddc19622ef59) in (session=66c72ac8-2423-fcad-e8fe-1b8214841318). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [ ]:
bucket='deloittesalarybucket'
prefix='multiplesmalldataset'
desired_size_in_bytes=1707008

Calculation started (calculation_id=48c72ac8-afe7-2cd2-044f-f6af02fac91d) in (session=66c72ac8-2423-fcad-e8fe-1b8214841318). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [ ]:
3/2

1.5

In [ ]:
def compaction():
    # Calculate partition details
    partition_file_num, partition_size_in_bytes = get_partition_num_and_size(bucket, prefix)
    average_file_size_in_bytes = partition_size_in_bytes / partition_file_num

    # Calculate optimal file count based on desired size
    optimal_file_num = math.floor(float(partition_size_in_bytes) / desired_size_in_bytes)
    print("optimal_file_num",optimal_file_num)
    # Check if average file size is less than desired size
    if average_file_size_in_bytes < desired_size_in_bytes:
        # Coalesce DataFrame to optimal file count
        df = spark.read.format('csv').option('header',True).load('s3://deloittesalarybucket/multiplesmalldataset/')
        df.repartition(optimal_file_num).write.mode('overwrite').format('csv').option('header',True). \
        save('s3://deloittesalarybucket/largedataset/')

Calculation started (calculation_id=92c72ac8-b9a1-fd78-95fe-d51429b62990) in (session=66c72ac8-2423-fcad-e8fe-1b8214841318). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [ ]:
compaction()

Calculation started (calculation_id=96c72ac8-c0eb-8455-3059-f2f3634cbdfa) in (session=66c72ac8-2423-fcad-e8fe-1b8214841318). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
optimal_file_num 1



In [ ]:
df2=spark.read.format('csv').option('header',True).load('s3://deloittesalarybucket/largedataset/')

Calculation started (calculation_id=e4c72ac8-eb3d-a947-40fc-ee0542225ace) in (session=66c72ac8-2423-fcad-e8fe-1b8214841318). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [ ]:
df2.count()

Calculation started (calculation_id=f0c72ac9-0181-987b-6cfe-a8b10bbdf396) in (session=66c72ac8-2423-fcad-e8fe-1b8214841318). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
17002

